In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
# use 2nd order or not
use_2nd = True
if use_2nd:
    fluid_input = 'inputs/fluidCgins_ho.cmd'
    fluid_output = 'outputs/fluid2ndos.pvd'
    solid_output = 'outputs/solid2ndps.pvd'
else:
    fluid_input = 'inputs/fluidCgins.cmd'
    fluid_output = 'outputs/fluidos.pvd'
    solid_output = 'outputs/solidos.pvd'

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
# create mapping
inodes = 64
f2s = np.empty(inodes, dtype=int)
s2f = np.empty(inodes, dtype=int)
# this is done offline!
f2s[0] = 0
f2s[-1] = 1
for i in range(inodes-2):
    f2s[i+1] = 62-i+1
s2f[0] = 0
s2f[1] = 63
for i in range(62):
    s2f[i+2] = 62-i

In [4]:
# fluid solver
interface_grid = 0
interface_side = 1
interface_axis = 0

solverF = Cgins()
solverF.read_cmd(fluid_input)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='outputs/dummy')
fluid_ofile = VtkFile(fluid_output)

In [5]:
# solid solver
fixed_temp_tag = 3
adiabatic_tag1 = 2
adiabatic_tag2 = 4
interface_tag = 5

kappa = 0.1
rho = 1.0
Cp = 0.485
T_bot = 1.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, transient=True)
solverS.output_file = solid_output
solverS.load_mesh('inputs/solid.msh')
solverS.set_initial_condition(0.0)
solverS.define_adiabatic_bd(tag=adiabatic_tag1)
solverS.define_adiabatic_bd(tag=adiabatic_tag2)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_flux_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
fluxF = InterfaceData(size=inodes, value=0.0)
fluxS = InterfaceData(size=inodes, value=0.0)
tempF = InterfaceData(size=inodes, value=0.0)
tempS = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=1e-6)

# maximum pc steps allowed
max_pc_steps = 200

In [7]:
# coupling timing information, use fixed time step for now
Tfinal = 1.0
N = 100
dt = Tfinal/N
t = 0.0

tf = 0.5*dt

In [8]:
# begin to couple
for step in range(N):
    t += dt
    
    # back up previous interface values
    # tempS.backup()
    
    # advance fluid
    solverF.advance(t=t, dt=dt)
    
    # get the time step flux
    fluxF.curr[:] = solverF.get_heat_flux_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis
    )
    
    # mapping
    fluxS.curr[:] = fluxF.curr[f2s]
    
    # update solid flux interface
    solverS.update_flux_interface(interface_tag, fluxS.curr)
    
    # advance solid
    solverS.advance(t=t, dt=dt)
    
    # retrieve solid interface temperature
    tempS.curr[:] = solverS.get_interface_temperature(tag=interface_tag)
    
    # time interpolation to half time step
    tempF.curr[:] = tempS.curr[s2f]
    
    # update fluid temperature interface
    solverF.put_temperature_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=tempF.curr
    )
    
    solverS.write()
    fluid_ofile.write(solverF, tf)
    